# Determine best batch processing configuration parameter values for Tierpsy tracker

To batch process worm videos using the Tierpsy tracker, you need a single parameter file.
I adopted a structured approach to determine the optimal parameters for batch processing multiple videos.
First, I manually adjusted the settings in the Tierpsy tracker GUI for each video to ensure I could consistently detect worms while minimizing background noise.
I aimed to keep the worms visible throughout the video without the mask obscuring the worm, which helps maintain worm identification across frames.

When I couldn't fully eliminate the background, I tweaked the settings to ensure it flickered enough so that it can be filtered out later as a non-worm entity because it wouldn't have enough frames where the "worm" was detected.

The parameters I used to optimize worm detection varied from video to video.

To streamline the selection of parameters for a batch of worms, I compiled all the optimized parameters into a single CSV file.
I ran the following script to parse the parameter JSON files from a specified directory and output them into a single CSV:

```{bash}
python scripts/parse_tierpsy_json_to_csv.py --directory ~/Downloads/RawVideos/ --output tierpsy_params_NGM.csv
```

Next, in this notebook, I analyze the data from the CSV to finalize the optimal parameter settings for processing multiple experiments.

The tables at the end of the notebook have the values I used for the Tierpsy tracker analysis.
I rounded down to the nearest whole number (9.22 -> 9) or to the nearest 100 (505.5555 -> 500).

In [30]:
library(tidyverse)

In [22]:
parameters <- read_csv("../tierpsy_params_NGM.csv", show_col_types = FALSE) %>%
  rename(parameter = `Parameter Name`)

simplify_column_names <- function(column_names) {
  column_names %>%
    str_remove(pattern = "/Users/taylorreiter/Downloads/RawVideos/NGM_Manual_tracking/") %>%
    str_remove(pattern = "/Users/taylorreiter/Downloads/RawVideos/NGM_manual_tracking_5h/") %>%
    str_remove(pattern = "/Point0000_ChannelJD_wormmotility_Seq0000") %>%
    str_remove(pattern = ".json")
    #str_extract(pattern = ".*(?=_Seq)") %>%
    #str_replace(pattern = "(_dogfilter)?$", replacement = function(x) ifelse(x == "_dogfilter", "-dogfilter", ""))
}

# Apply the function to the column names
names(parameters) <- simplify_column_names(names(parameters))

In [37]:
parameters %>%
  select(parameter, ends_with("dogfilter")) %>%
  filter(parameter %in% c("mask_min_area", "thresh_C", "thresh_block_size", "dilation_size")) %>%
  pivot_longer(cols = ends_with("dogfilter"), names_to = "image", values_to = "value") %>%
  group_by(parameter) %>%
  summarize(mean_value = mean(as.numeric(value)))

parameter,mean_value
<chr>,<dbl>
dilation_size,9.00000
mask_min_area,505.55556
thresh_C,10.33333
thresh_block_size,51.22222


In [40]:
parameters %>%
  select(parameter, !ends_with("dogfilter")) %>%
  filter(parameter %in% c("mask_min_area", "thresh_C", "thresh_block_size", "dilation_size")) %>%
  pivot_longer(cols = starts_with("2024"), names_to = "image", values_to = "value") %>%
  group_by(parameter) %>%
  summarize(mean_value = mean(as.numeric(value)))

parameter,mean_value
<chr>,<dbl>
dilation_size,9.222222
mask_min_area,922.222222
thresh_C,15.111111
thresh_block_size,52.444444


In [41]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/wormmotility/lib/libopenblasp-r0.3.27.dylib;  LAPACK version 3.12.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.3 forcats_1.0.0   stringr_1.5.1   dplyr_1.1.4    
 [5] purrr_1.0.2     readr_2.1.5     tidyr_1.3.1     tibble_3.2.1   
 [9] ggplot2_3.5.1   tidyverse_2.0.0

loaded via a namespace (and not attached):
 [1] bit_4.0.5         gtable_0.3.5      jsonlite_1.8.8    compiler_4.3.3   
 [5] crayon_1.5.3      tidyselect_1.2.1  IRdisplay_1.1     parallel_4.3.3   
 [9] scales_1.3.0      uuid_1.2-0        fastmap_1.2.0     IRkernel_1.3.2   
[13] R6_2.5.1          generics_0.1.3    